In [ ]:

from scipy.stats import wilcoxon
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import networkx as nx
import bct
import os
import glob
%matplotlib widget

Info_Pacientes = os.path.join('subject_info.csv')
Datos_Procesados = r"D:\GitHub\Neurociencias-2026-1\Neurociencias-2026-1\S06_Proyecto\Proyecto_Epilepsia\datos_procesados"
Coordinadas_csv = "MI_coordinates.csv"


In [ ]:
def calcular_metricas_csv_corregido(file_path):
    try:
        #Primero cargaré las matrices de adyacencia que cree en otro script para que el procesamiento fuera más rápido y quitaré todos los Nan
        Matriz_Adyacencia = pd.read_csv(file_path, index_col=0).select_dtypes(include=[np.number])
        Matriz_Adyacencia = Matriz_Adyacencia.fillna(0)
        W = np.abs(Matriz_Adyacencia.values)
        np.fill_diagonal(W, 0)
        if W.shape[0] < 5: return None
        with np.errstate(divide='ignore'): L_mat = 1.0 / (W + 1e-9)
        np.fill_diagonal(L_mat, 0)
        C = np.mean(bct.clustering_coef_wu(W))

        #Aqui calcularé la modularidad
        _, Q = bct.community_louvain(W)

        Strength = np.mean(bct.strengths_und(W))

        D = bct.distance_wei(L_mat)[0]
        valid_D = D[np.triu_indices(W.shape[0], k=1)]
        L_avg = np.mean(valid_D[np.isfinite(valid_D)]) if len(valid_D) > 0 else 0

        return {'C_avg': C, 'L_avg': L_avg, 'Q': Q, 'Strength_avg': Strength}
    except Exception as e:
        return None


Archivos_csv = glob.glob(os.path.join(Datos_Procesados, "PN*.csv"))
results = []

for file_path in Archivos_csv:
    if os.path.getsize(file_path) > 500 * 1024: continue

    Nombre_archivo = os.path.basename(file_path)
    parts = Nombre_archivo.replace('.csv', '').split('_')
    if len(parts) < 2: continue

    Numero_paciente = parts[0]
    Condicion = "BASAL" if "BASAL" in Nombre_archivo else "CRISIS"

    m = calcular_metricas_csv_corregido(file_path)
    if m: results.append({'Numero paciente': Numero_paciente, 'condicion': Condicion, 'source': Nombre_archivo, **m})

if results:
    df = pd.DataFrame(results)
    df.to_csv("Resultados1.csv", index=False)
    display(df.head())
else:
    print (None)

In [ ]:
if os.path.exists("Resultados1.csv"):
    df = pd.read_csv("Resultados1.csv")

    # Aqui hago un promedio de las criis por cada paciente para así comparar entre estados basales y de crisis
    df_avg = df.groupby(['Numero paciente', 'condicion'])[['C_avg', 'L_avg', 'Q', 'Strength_avg']].mean().reset_index()

    metrics = {'Q': 'Modularidad', 'C_avg': 'Clustering', 'L_avg': 'Eficiencia (L)', 'Strength_avg': 'Fuerza Global'}

    plt.figure(figsize=(16, 5))
    sns.set(style="whitegrid")

    for i, (col, label) in enumerate(metrics.items()):
        plt.subplot(1, 4, i+1)
        sns.violinplot(x='condicion', y=col, data=df_avg, palette=['#a1c9f4', '#ff9f9b'], inner=None)
        sns.stripplot(x='condicion', y=col, data=df_avg, color=".2", alpha=0.6)

        piv = df_avg.pivot(index='Numero paciente', columns='condicion', values=col).dropna()
        for idx in piv.index:
            plt.plot([0, 1], [piv.loc[idx, 'BASAL'], piv.loc[idx, 'CRISIS']], color='gray', alpha=0.3)

        plt.title(label, fontweight='bold')
        plt.xlabel("")
        plt.ylabel("")

    plt.tight_layout()
    plt.show()

    print(f"{'Métrica':<15} | {'p-value':<10} | {'Conclusión'}")
    print("-" * 45)
    pivot = df_avg.pivot(index='Numero paciente', columns='condicion', values=list(metrics.keys())).dropna()
    for m in metrics.keys():
        stat, p = wilcoxon(pivot[(m, 'BASAL')], pivot[(m, 'CRISIS')])
        sig = "Significativo" if p < 0.05 else "No significativo."
        print(f"{metrics[m]:<15} | {p:.4f}     | {sig}")
else:
    print (None)

In [ ]:

def dibujar_cerebro_denso_con_fondo(csv_path, coords_path, ax, title):
    try:

        Matriz_Adyacencia = pd.read_csv(csv_path, index_col=0).select_dtypes(include=[np.number]).fillna(0)
        coordenadas = pd.read_csv(coords_path)

        nombres_limpios = [n.replace('EEG', '').replace('.', '').strip() for n in Matriz_Adyacencia.index]
        Matriz_Adyacencia.index = nombres_limpios
        Matriz_Adyacencia.columns = nombres_limpios

        coordenadas['canal'] = coordenadas['canal'].astype(str).str.replace('.', '').str.strip()
        coordenadas.set_index('canal', inplace=True)

        nodes = [n for n in Matriz_Adyacencia.index if n in coordenadas.index]
        if not nodes: return

        Matriz_Adyacencia = Matriz_Adyacencia.loc[nodes, nodes]
        coordenadas = coordenadas.loc[nodes]

        W = np.abs(Matriz_Adyacencia.values)
        np.fill_diagonal(W, 0)
        th = np.percentile(W, 50)
        G = nx.from_numpy_array((W > th).astype(int))
        mapping = {i: n for i, n in enumerate(nodes)}
        G = nx.relabel_nodes(G, mapping)

        try: comunidades = list(nx.community.greedy_modularity_communities(G))
        except: comunidades = [set(G.nodes)]

        node_colors = {}
        cmap = plt.get_cmap('Set1')
        for i, com in enumerate(comunidades):
            color = cmap(i % 9)
            for node in com: node_colors[node] = color

        degree = dict(G.degree())
        hub_nombre = max(degree, key=degree.get)

        for u, v in G.edges:
            color_u, color_v = node_colors.get(u, 'gray'), node_colors.get(v, 'gray')
            if np.allclose(color_u, color_v):
                ec, alpha, width = color_u, 0.5, 1.2
            else:
                ec, alpha, width = 'lightgray', 0.15, 0.6

            p1, p2 = coordenadas.loc[u], coordenadas.loc[v]
            ax.plot([p1.x, p2.x], [p1.y, p2.y], [p1.z, p2.z], c=ec, alpha=alpha, linewidth=width)

        x, y, z = coordenadas['x'], coordenadas['y'], coordenadas['z']
        c_list = [node_colors.get(n, 'gray') for n in nodes]
        s_list = [180 if n == hub_nombre else 50 for n in nodes]

        ax.scatter(x, y, z, s=s_list, c=c_list, alpha=1.0, edgecolors='k', linewidth=0.6)

        for n in nodes:
            fw, fs = ('bold', 9) if n == hub_nombre else ('normal', 7)

            ax.text(coordenadas.loc[n].x, coordenadas.loc[n].y, coordenadas.loc[n].z + 0.05, n, fontsize=fs, fontweight=fw)

        ax.set_title(f"{title}\nHub: {hub_nombre}", fontsize=12, fontweight='bold')

        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')

        ax.view_init(elev=35, azim=135)

    except Exception as e:
        print(f"Error {title}: {e}")

#para no imprimir todos los grafos 3D y que no se sature la memoria aqui solo es cambiar el numero de paciente despues del PN
file_b = os.path.join(Datos_Procesados, "PN10_BASAL.csv")
file_c = os.path.join(Datos_Procesados, "PN10_CRISIS_1.csv")

if os.path.exists(file_b) and os.path.exists(file_c):
    fig = plt.figure(figsize=(12, 7)) # Tamaño balanceado

    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    dibujar_cerebro_denso_con_fondo(file_b, Coordinadas_csv, ax1, "ESTADO BASAL")

    ax2 = fig.add_subplot(1, 2, 2, projection='3d')
    dibujar_cerebro_denso_con_fondo(file_c, Coordinadas_csv, ax2, "DURANTE CRISIS")

    plt.tight_layout()
    plt.show()

In [ ]:
def calcular_sigma_y_metricas(file_path):
    try:
        df_mat = pd.read_csv(file_path, index_col=0).select_dtypes(include=[np.number]).fillna(0)
        W = np.abs(df_mat.values)
        np.fill_diagonal(W, 0)

        if W.shape[0] < 5: return None

        C_real = np.mean(bct.clustering_coef_wu(W))
        _, Q = bct.community_louvain(W)
        Strength = np.mean(bct.strengths_und(W))

        # Eficiencia se representa con L
        with np.errstate(divide='ignore'):
            L_mat = 1.0 / (W + 1e-9)
        np.fill_diagonal(L_mat, 0)
        D = bct.distance_wei(L_mat)[0]
        valid_D = D[np.triu_indices(W.shape[0], k=1)]
        L_real = np.mean(valid_D[np.isfinite(valid_D)]) if len(valid_D) > 0 else 0

        # Sigma es el coeficiente de mundo pequeño
        th = np.percentile(W, 80)
        A = (W > th).astype(int)
        G = nx.from_numpy_array(A)

        G_rand = nx.gnm_random_graph(len(G.nodes), len(G.edges))

        C_bin = nx.average_clustering(G)
        L_bin = nx.average_shortest_path_length(G) if nx.is_connected(G) else np.nan

        C_rand = nx.average_clustering(G_rand)
        L_rand = nx.average_shortest_path_length(G_rand) if nx.is_connected(G_rand) else np.nan

        if L_bin > 0 and L_rand > 0 and C_rand > 0:
            sigma = (C_bin / C_rand) / (L_bin / L_rand)
        else:
            sigma = np.nan

        pesos = W[np.triu_indices(W.shape[0], k=1)]
        peso_medio = np.mean(pesos)

        return {
            'C_avg': C_real, 'L_avg': L_real, 'Q': Q, 'Strength_avg': Strength,
            'Sigma': sigma, 'Peso_medio': peso_medio
        }

    except Exception as e:
        return None


if os.path.exists(Info_Pacientes):
    df_info = pd.read_csv(Info_Pacientes)


    df_info.columns = df_info.columns.str.strip()
    df_info['patient_id'] = df_info['patient_id'].astype(str).str.strip()


    col_crisis = 'seizure'
    if 'seizure' not in df_info.columns:
        posibles = [c for c in df_info.columns if 'seiz' in c.lower() or 'crisis' in c.lower()]
        if posibles:
            col_crisis = posibles[0]

        else:

            col_crisis = None

    col_edad = 'años'
    if 'años' not in df_info.columns:
        posibles_edad = [c for c in df_info.columns if 'age' in c.lower() or 'edad' in c.lower()]
        if posibles_edad: col_edad = posibles_edad[0]
else:
    df_info = pd.DataFrame()
    col_crisis = None


Archivos_csv = glob.glob(os.path.join(Datos_Procesados, "PN*.csv"))
results = []

for file_path in Archivos_csv:
    if os.path.getsize(file_path) > 500 * 1024: continue

    Nombre_archivo = os.path.basename(file_path)
    parts = Nombre_archivo.replace('.csv', '').split('_')
    if len(parts) < 2: continue

    pid = parts[0]
    cond = "BASAL" if "BASAL" in Nombre_archivo else "CRISIS"

    m = calcular_sigma_y_metricas(file_path)
    if m:

        tipo_crisis = "Desconocido"
        edad = np.nan

        if not df_info.empty:
            meta = df_info[df_info['patient_id'] == pid]
            if not meta.empty:
                if col_crisis: tipo_crisis = meta[col_crisis].values[0]
                if col_edad: edad = meta[col_edad].values[0]

        results.append({
            'Numero paciente': pid, 'condicion': cond, 'source': Nombre_archivo,
            'Tipo_Crisis': tipo_crisis, 'Edad': edad,
            **m
        })
if results:
    df_final = pd.DataFrame(results)
    df_final.to_csv("Resultados2.csv", index=False)

#Estos son los ID de los pacientes, para tenerlos precentes al cambiar la siguiente linea que nos muestra los datos calculados
    print("Pacientes procesados:", df_final['Numero paciente'].unique())
    Paciente = 'PN03'
    df_paciente = df_final[df_final['Numero paciente'] == Paciente]
    display(df_paciente)

else:
    print(None)

In [ ]:

if os.path.exists("Resultados2.csv"):
    df = pd.read_csv("Resultados2.csv")

    def obtener_matriz_promedio(condicion):
        matrices = []
        files = df[df['condicion'] == condicion]['source'].tolist()
        for f in files[:10]:  # Usamos una muestra para no saturar memoria
            path = os.path.join(Datos_Procesados, f)
            try:
                m = pd.read_csv(path, index_col=0).select_dtypes(include=[np.number]).fillna(0).values
                # Normalizar tamaño (recortar a 20 canales comunes para promediar)
                if m.shape[0] >= 20: matrices.append(m[:20, :20])
            except:
                pass
        if matrices: return np.mean(matrices, axis=0)
        return None


    mat_basal = obtener_matriz_promedio("BASAL")
    mat_crisis = obtener_matriz_promedio("CRISIS")

    if mat_basal is not None:
        fig, ax = plt.subplots(1, 2, figsize=(16, 6))
        sns.heatmap(mat_basal, ax=ax[0], cmap="magma", vmin=0, vmax=0.8)
        ax[0].set_title("Matriz Promedio: ESTADO BASAL")

        sns.heatmap(mat_crisis, ax=ax[1], cmap="magma", vmin=0, vmax=0.8)
        ax[1].set_title("Matriz Promedio: CRISIS")
        plt.show()

    # --- B. DISTRIBUCIÓN DE DATOS (Pesos) ---
    plt.figure(figsize=(10, 5))
    sns.kdeplot(data=df, x='Peso_Mean', hue='condicion', fill=True, palette=['blue', 'red'])
    plt.title("Distribución de la Fuerza de Conexión (Pesos Promedio)")
    plt.xlabel("Peso Promedio de Conexiones")
    plt.show()

In [ ]:


# --- 1. CARGAR DATOS ---
if os.path.exists("Resultados2.csv"):
    df = pd.read_csv("Resultados2.csv")

    # LIMPIEZA AGRESIVA PARA EVITAR EL BUG DE SEABORN
    # 1. Eliminar filas vacías en columnas clave
    df_clean = df.dropna(subset=['Edad', 'Tipo_Crisis']).copy()

    # 2. Convertir a String explícito (Esto arregla el error de 'boxprops')
    df_clean['Tipo_Crisis'] = df_clean['Tipo_Crisis'].astype(str)

    # 3. Crear grupos de edad
    df_clean['Grupo_Edad'] = pd.cut(df_clean['Edad'], bins=[0, 30, 50, 100],
                                    labels=['Joven (<30)', 'Adulto (30-50)', 'Mayor (>50)'])
    # Convertir también a string para seguridad
    df_clean['Grupo_Edad'] = df_clean['Grupo_Edad'].astype(str)

    # 4. Filtrar top crisis
    top_crisis = df_clean['Tipo_Crisis'].value_counts().index[:3]
    df_crisis_filt = df_clean[df_clean['Tipo_Crisis'].isin(top_crisis)].copy()


    # --- 2. GRAFICAR ---
    metrics_to_plot = ['Sigma', 'Q', 'L_avg', 'Strength_avg']
    titles = ['Small-World (Sigma)', 'Modularidad', 'Eficiencia (L)', 'Fuerza']

    sns.set(style="whitegrid", context="talk")

    try:
        # GRÁFICO A: TIPO DE CRISIS
        if not df_crisis_filt.empty:
            plt.figure(figsize=(16, 10))
            plt.suptitle("Comparación por TIPO DE CRISIS", fontsize=16, y=1.02)

            for i, metric in enumerate(metrics_to_plot):
                plt.subplot(2, 2, i + 1)
                # dropna() aquí mismo asegura que no pasen nulos al plot
                sns.boxplot(x='Tipo_Crisis', y=metric, hue='condicion', data=df_crisis_filt.dropna(subset=[metric]),
                            palette='Set2')
                plt.title(titles[i])
                plt.xlabel("")
            plt.tight_layout()
            plt.show()

        # GRÁFICO B: EDAD
        if not df_clean.empty:
            plt.figure(figsize=(16, 10))
            plt.suptitle("Comparación por GRUPO DE EDAD", fontsize=16, y=1.02)

            # Ordenar las etiquetas de edad para que salgan bien en el gráfico
            order_edad = ['Joven (<30)', 'Adulto (30-50)', 'Mayor (>50)']

            for i, metric in enumerate(metrics_to_plot):
                plt.subplot(2, 2, i + 1)
                sns.boxplot(x='Grupo_Edad', y=metric, hue='condicion', data=df_clean.dropna(subset=[metric]),
                            order=order_edad, palette='coolwarm')
                plt.title(titles[i])
                plt.xlabel("")
            plt.tight_layout()
            plt.show()

    except Exception as e:
        print(f"{e}")

else:
    print(None)